## Task:

Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size

In [14]:
import os
import pandas as pd
import numpy as np

In [20]:
%%time
df_sample = pd.read_csv('nyt-comments-2020.csv')

<timed exec>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 1min 3s


In [25]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986461 entries, 0 to 4986460
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   commentID              int64  
 1   status                 object 
 2   commentSequence        int64  
 3   userID                 int64  
 4   userDisplayName        object 
 5   userLocation           object 
 6   userTitle              object 
 7   commentBody            object 
 8   createDate             object 
 9   updateDate             object 
 10  approveDate            object 
 11  recommendations        int64  
 12  replyCount             int64  
 13  editorsSelection       bool   
 14  parentID               float64
 15  parentUserDisplayName  object 
 16  depth                  int64  
 17  commentType            object 
 18  trusted                int64  
 19  recommendedFlag        int64  
 20  permID                 int64  
 21  isAnonymous            bool   
 22  articleID         

In [5]:
%%time
from dask import dataframe as dd
dask_df = dd.read_csv('nyt-comments-2020.csv')

Wall time: 2.1 s


In [7]:
%%time
import ray
ray_df = ray.data.read_csv('nyt-comments-2020.csv')


2023-06-09 18:09:10,263	INFO worker.py:1636 -- Started a local Ray instance.
2023-06-09 18:09:15,042	WARNING read_api.py:393 -- ⚠️  The blocks of this dataset are estimated to be 5.0x larger than the target block size of 512 MiB. This may lead to out-of-memory errors during processing. Consider reducing the size of input files or using `.repartition(n)` to increase the number of dataset blocks.
2023-06-09 18:09:15,051	WARNING dataset.py:253 -- Important: Ray Data requires schemas for all datasets in Ray 2.5. This means that standalone Python objects are no longer supported. In addition, the default batch format is fixed to NumPy. To revert to legacy behavior temporarily, set the environment variable RAY_DATA_STRICT_MODE=0 on all cluster processes.

Learn more here: https://docs.ray.io/en/master/data/faq.html#migrating-to-strict-mode


Wall time: 11 s


In [8]:
%%time
import modin.pandas as pd
ray.shutdown()
ray.init()

mod_df = pd.read_csv('nyt-comments-2020.csv')

2023-06-09 18:09:44,597	INFO worker.py:1636 -- Started a local Ray instance.


Wall time: 45.8 s


In [6]:
df = dd.read_csv('nyt-comments-2020.csv')

In [12]:
len(df.columns)

23

In [7]:
# remove white spaces in the column names
df.columns = df.columns.str.replace(' ', '')

In [8]:
# remove special characters
df.columns=df.columns.str.replace('[#,@,&]','')

C:\Users\Drmoke\AppData\Local\Temp\ipykernel_27636\3040113392.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns=df.columns.str.replace('[#,@,&]','')


In [9]:
columns = df.columns
columns

Index(['commentID', 'status', 'commentSequence', 'userID', 'userDisplayName',
       'userLocation', 'userTitle', 'commentBody', 'createDate', 'updateDate',
       'approveDate', 'recommendations', 'replyCount', 'editorsSelection',
       'parentID', 'parentUserDisplayName', 'depth', 'commentType', 'trusted',
       'recommendedFlag', 'permID', 'isAnonymous', 'articleID'],
      dtype='object')

### We can see dask has the quickest processing time, following by ray and modin, pandas has the lowest processing time

In [36]:
%%writefile utility.py
# Use the utility code template
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [11]:
%%writefile nyt.yaml
file_type: csv
dataset_name: nyt-comments-2020
file_name: nyt-comments-2020
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - commentID
    - status
    - commentSequence
    - userID
    - userDisplayName
    - userLocation
    - userTitle
    - commentBody
    - createDate
    - updateDate
    - approveDate
    - recommendations
    - replyCount
    - editorsSelection
    - parentID
    - parentUserDisplayName
    - depth
    - commentType
    - trusted
    - recommendedFlag
    - permID
    - isAnonymous
    - articleID

Overwriting nyt.yaml


In [2]:
import utility as util
config_data = util.read_config_file("nyt.yaml")

In [3]:
config_data

{'file_type': 'csv',
 'dataset_name': 'nyt-comments-2020',
 'file_name': 'nyt-comments-2020',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['commentID',
  'status',
  'commentSequence',
  'userID',
  'userDisplayName',
  'userLocation',
  'userTitle',
  'commentBody',
  'createDate',
  'updateDate',
  'approveDate',
  'recommendations',
  'replyCount',
  'editorsSelection',
  'parentID',
  'parentUserDisplayName',
  'depth',
  'commentType',
  'trusted',
  'recommendedFlag',
  'permID',
  'isAnonymous',
  'articleID']}

In [4]:
# use dask to read dataset file
import numpy as np
from dask import dataframe as dd
df = dd.read_csv('nyt-comments-2020.csv',delimiter=',', dtype={'userTitle': 'object'})

In [26]:
df.head()

,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,commentBody,createDate,updateDate,...,editorsSelection,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID
0,104387472,approved,104387472,60215558,magicisnotreal,earth,NaN,Here is something I think is fraudulent that v...,2020-01-01 01:05:46,2020-01-01 08:13:39,...,False,NaN,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,104387873,approved,104387873,65691034,JD,Elko,NaN,@magicisnotreal I have used my VA loan option...,2020-01-01 01:52:25,2020-01-01 20:55:19,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
2,104387976,approved,104387976,65110053,ebmem,"Memphis, TN",NaN,@magi\n\nWhy would someone take out a VA loan ...,2020-01-01 02:06:05,2020-01-01 20:55:35,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
3,104390628,approved,104390628,60215558,magicisnotreal,earth,NaN,@JD\nOut here in the Alabama of the PNW they w...,2020-01-01 14:38:50,2020-01-01 20:56:46,...,False,104387873.0,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
4,104391463,approved,104391463,65691034,JD,Elko,NaN,@magicisnotreal just a guess but I doubt that...,2020-01-01 16:23:14,2020-01-01 16:25:57,...,False,104390628.0,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...


In [7]:
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'

In [8]:
df_con = pd.read_csv(source_file,config_data['inbound_delimiter'])
df_con.head()

C:\Users\Drmoke\AppData\Local\Temp\ipykernel_12020\3782064743.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_con = pd.read_csv(source_file,config_data['inbound_delimiter'])
C:\Users\Drmoke\AppData\Local\Temp\ipykernel_12020\3782064743.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_con = pd.read_csv(source_file,config_data['inbound_delimiter'])


,commentID,status,commentSequence,userID,userDisplayName,userLocation,userTitle,commentBody,createDate,updateDate,...,editorsSelection,parentID,parentUserDisplayName,depth,commentType,trusted,recommendedFlag,permID,isAnonymous,articleID
0,104387472,approved,104387472,60215558,magicisnotreal,earth,NaN,Here is something I think is fraudulent that v...,2020-01-01 01:05:46,2020-01-01 08:13:39,...,False,NaN,NaN,1,comment,0,0,104387472,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
1,104387873,approved,104387873,65691034,JD,Elko,NaN,@magicisnotreal I have used my VA loan option...,2020-01-01 01:52:25,2020-01-01 20:55:19,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387873,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
2,104387976,approved,104387976,65110053,ebmem,"Memphis, TN",NaN,@magi\n\nWhy would someone take out a VA loan ...,2020-01-01 02:06:05,2020-01-01 20:55:35,...,False,104387472.0,magicisnotreal,2,userReply,0,0,104387976,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
3,104390628,approved,104390628,60215558,magicisnotreal,earth,NaN,@JD\nOut here in the Alabama of the PNW they w...,2020-01-01 14:38:50,2020-01-01 20:56:46,...,False,104387873.0,magicisnotreal,2,userReply,0,0,104390628,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...
4,104391463,approved,104391463,65691034,JD,Elko,NaN,@magicisnotreal just a guess but I doubt that...,2020-01-01 16:23:14,2020-01-01 16:25:57,...,False,104390628.0,magicisnotreal,2,userReply,0,0,104391463,False,nyt://article/69a7090b-9f36-569e-b5ab-b0ba5bb3...


In [10]:
df_con.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986461 entries, 0 to 4986460
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   commentid              int64  
 1   status                 object 
 2   commentsequence        int64  
 3   userid                 int64  
 4   userdisplayname        object 
 5   userlocation           object 
 6   usertitle              object 
 7   commentbody            object 
 8   createdate             object 
 9   updatedate             object 
 10  approvedate            object 
 11  recommendations        int64  
 12  replycount             int64  
 13  editorsselection       bool   
 14  parentid               float64
 15  parentuserdisplayname  object 
 16  depth                  int64  
 17  commenttype            object 
 18  trusted                int64  
 19  recommendedflag        int64  
 20  permid                 int64  
 21  isanonymous            bool   
 22  articleid         

In [15]:
util.col_header_val(df_con,config_data)

column name and column length validation passed


1

In [16]:
print("columns of files are:" ,df_con.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['commentid', 'status', 'commentsequence', 'userid', 'userdisplayname',
       'userlocation', 'usertitle', 'commentbody', 'createdate', 'updatedate',
       'approvedate', 'recommendations', 'replycount', 'editorsselection',
       'parentid', 'parentuserdisplayname', 'depth', 'commenttype', 'trusted',
       'recommendedflag', 'permid', 'isanonymous', 'articleid'],
      dtype='object')
columns of YAML are: ['commentID', 'status', 'commentSequence', 'userID', 'userDisplayName', 'userLocation', 'userTitle', 'commentBody', 'createDate', 'updateDate', 'approveDate', 'recommendations', 'replyCount', 'editorsSelection', 'parentID', 'parentUserDisplayName', 'depth', 'commentType', 'trusted', 'recommendedFlag', 'permID', 'isAnonymous', 'articleID']


In [17]:
if util.col_header_val(df_con,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


In [7]:
import datetime
import csv
import gzip

# Write csv in gz format in pipe separated text file (|)
df = pd.read_csv('nyt-comments-2020.csv', engine='python')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986461 entries, 0 to 4986460
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   commentID              int64  
 1   status                 object 
 2   commentSequence        int64  
 3   userID                 int64  
 4   userDisplayName        object 
 5   userLocation           object 
 6   userTitle              object 
 7   commentBody            object 
 8   createDate             object 
 9   updateDate             object 
 10  approveDate            object 
 11  recommendations        int64  
 12  replyCount             int64  
 13  editorsSelection       bool   
 14  parentID               float64
 15  parentUserDisplayName  object 
 16  depth                  int64  
 17  commentType            object 
 18  trusted                int64  
 19  recommendedFlag        int64  
 20  permID                 int64  
 21  isAnonymous            bool   
 22  articleID         

In [12]:
df.to_csv('nyt-comments-2020.csv.gz',
          sep='|',
          index=False,
          compression='gzip')

In [15]:
os.path.getsize('nyt-comments-2020.csv.gz')

1027737940